### Do stuff with curated bouts
- Get the wav files
- Get out the pandas wiht the metadata
- fore each bout make a json file compatible with the avgn repository

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal
import json

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2021-08-12 11:18:27,656 root         INFO     Running on pakhi


In [2]:
from ceciestunepipe.file import bcistructure as et

reload(et)
sess_par = {'bird': 's_b1253_21',
           'sess': 'dummy',
           'ephys_software': 'alsa',
           'bouts_detector': 'bouts_ceciestunepipe',
           'bouts_file': 'bout_checked.pickle'
}


bird_sessions = et.list_sessions(sess_par['bird'], ephys_software='alsa')

### list bird sessions with curated bouts

In [3]:
def list_bird_bout_files(sess_par: dict):
    # list all the sessions for the bird
    bird_sessions = et.list_sessions(sess_par['bird'], ephys_software=sess_par['ephys_software'])
    bird_sessions.sort()

    
    # for each session, look whether there is a bout_checked_file
    all_bout_path = []
    for sess in bird_sessions:
        exp_struct = et.get_exp_struct(sess_par['bird'], sess, ephys_software=sess_par['ephys_software'])
        
        bouts_folder = os.path.join(exp_struct['folders']['derived'], sess_par['bouts_detector'])
        bouts_file_path = os.path.join(bouts_folder, sess_par['bouts_file'])
        #logger.info(bouts_file_path)
        if os.path.isfile(bouts_file_path):
            all_bout_path.append(bouts_file_path)
    return all_bout_path
            
bird_bout_file_list = list_bird_bout_files(sess_par)
        

In [4]:
bird_bout_file_list

['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-06/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-09/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-12/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-15/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-16/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-17/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-18/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/der

In [5]:
from tqdm.notebook import trange, tqdm

In [6]:
def load_all_bout_pandas(sess_par: dict):
    # list all the bout files in the session
    path_list = list_bird_bout_files(sess_par)
    # load them all and concatenate into one single pandas dataframe
    pd_list = []
    logger.info('loading all pandas {} for bird {}'.format(sess_par['bouts_file'], sess_par['bird']))
    for i_p, one_path in enumerate(tqdm(path_list)):
        one_pd = pd.read_pickle(one_path)
        pd_list.append(one_pd)
    
    return pd.concat(pd_list).reset_index(drop=True)

bird_bout_pd = load_all_bout_pandas(sess_par)
        

2021-08-12 11:18:52,055 root         INFO     loading all pandas bout_checked.pickle for bird s_b1253_21


  0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
bird_bout_pd.head(3)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,224595.0,251655.0,10780560.0,12079440.0,"[2.194746139749094, 3.850394476655695, 2.16753...",3.837231,104.672704,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,27060.0,"[[224595, 224700], [226320, 226465], [227510, ...",73.0,"[1, 10, 18, 32, 41, 348, 356, 364, 373, 386, 3...",240.0,112.750000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, -2, -1, -1,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
1,924555.0,945395.0,44378640.0,45378960.0,"[2.6427765346156753, 3.1863976015258806, 4.622...",3.837231,115.114813,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,20840.0,"[[924555, 924635], [925235, 925290], [925315, ...",82.0,"[7, 14, 31, 76, 145, 155, 162, 173, 201, 239, ...",233.0,89.442060,"[-5, -4, -4, -3, -4, -4, -4, -6, -4, -2, -3, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
2,1041975.0,1071595.0,50014800.0,51436560.0,"[26.3627577571065, 27.576856238944778, 24.7911...",3.837231,164.391799,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,29620.0,"[[1041975, 1042015], [1042045, 1042075], [1042...",93.0,"[1, 16, 57, 126, 162, 170, 187, 232, 243, 293,...",324.0,91.419753,"[3, 3, 1, 1, 2, 1, 1, 1, 3, 4, 3, 3, 2, 2, 1, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False


##### remove all the 'confusing' or noise ('!bout_check')

In [9]:
only_bout_pd = bird_bout_pd.drop(bird_bout_pd[(bird_bout_pd['confusing']==True) | (bird_bout_pd['bout_check']==False)].index)

In [10]:
only_bout_pd.head(3)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,224595.0,251655.0,10780560.0,12079440.0,"[2.194746139749094, 3.850394476655695, 2.16753...",3.837231,104.672704,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,27060.0,"[[224595, 224700], [226320, 226465], [227510, ...",73.0,"[1, 10, 18, 32, 41, 348, 356, 364, 373, 386, 3...",240.0,112.750000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, -2, -1, -1,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
1,924555.0,945395.0,44378640.0,45378960.0,"[2.6427765346156753, 3.1863976015258806, 4.622...",3.837231,115.114813,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,20840.0,"[[924555, 924635], [925235, 925290], [925315, ...",82.0,"[7, 14, 31, 76, 145, 155, 162, 173, 201, 239, ...",233.0,89.442060,"[-5, -4, -4, -3, -4, -4, -4, -6, -4, -2, -3, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
2,1041975.0,1071595.0,50014800.0,51436560.0,"[26.3627577571065, 27.576856238944778, 24.7911...",3.837231,164.391799,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,29620.0,"[[1041975, 1042015], [1042045, 1042075], [1042...",93.0,"[1, 16, 57, 126, 162, 170, 187, 232, 243, 293,...",324.0,91.419753,"[3, 3, 1, 1, 2, 1, 1, 1, 3, 4, 3, 3, 2, 2, 1, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False


In [52]:
only_bout_pd.dropna(inplace=True)

2021-08-12 12:37:14,431 numexpr.utils INFO     Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-08-12 12:37:14,431 numexpr.utils INFO     NumExpr defaulting to 8 threads.


### save it somewhere

In [11]:
bout_folder = '/mnt/sphere/speech_bci/analysis/bout/s_b1253_21'
bout_file = 'check_bouts.pickle'

only_bout_pd.to_pickle(os.path.join(bout_folder, bout_file))

In [12]:
os.path.join(bout_folder, bout_file)

'/mnt/sphere/speech_bci/analysis/bout/s_b1253_21/check_bouts.pickle'

### save each bout to a wav file

In [13]:
one_bout = only_bout_pd.iloc[0]
one_bout

start_ms                                                   224595.0
end_ms                                                     251655.0
start_sample                                             10780560.0
end_sample                                               12079440.0
p_step            [2.194746139749094, 3.850394476655695, 2.16753...
rms_p                                                      3.837231
peak_p                                                   104.672704
bout_check                                                     True
file              /mnt/sphere/speech_bci/raw_data/s_b1253_21/202...
len_ms                                                      27060.0
syl_in            [[224595, 224700], [226320, 226465], [227510, ...
n_syl                                                          73.0
peaks_p           [1, 10, 18, 32, 41, 348, 356, 364, 373, 386, 3...
n_peaks                                                       240.0
l_p_ratio                                       

In [14]:
np.where(only_bout_pd['valid'] == False)

(array([], dtype=int64),)

In [15]:
one_bout.index

Index(['start_ms', 'end_ms', 'start_sample', 'end_sample', 'p_step', 'rms_p',
       'peak_p', 'bout_check', 'file', 'len_ms', 'syl_in', 'n_syl', 'peaks_p',
       'n_peaks', 'l_p_ratio', 'waveform', 'valid_waveform', 'valid',
       'spectrogram', 'confusing'],
      dtype='object')

In [16]:
fpath, fname = os.path.split(one_bout['file'])

fpath.split(os.path.sep)

['',
 'mnt',
 'sphere',
 'speech_bci',
 'raw_data',
 's_b1253_21',
 '2021-06-06',
 'alsa']

In [73]:
wavfile.write?

In [21]:
def bout_to_json(bout_dict, json_path):
    os.makedirs(os.path.split(json_path)[0], exist_ok=True)
    with open(json_path, 'w') as json_file:
        json.dump(bout_dict, json_file)

In [58]:
bout_wav_folder = os.path.join(bout_folder, 'bout_wav')

def bout_to_wav(bout_s: pd.Series, s_f=48000, border_ms=1000):
    # trim the edges
    border_samples = int(s_f * border_ms * 0.001)
    # get the wav for the series (trim the edges)
    #x = bout_s['waveform'][border_samples: -border_samples]
    x = bout_s['waveform']
    #define wav file
    path_parts, path_name = os.path.split(bout_s['file'])
    [bird, sess] = path_parts.split(os.path.sep)[5:7]
    file = path_name.split('.wav')[0]
    
    out_fname = '{}-{}-{}-{}.wav'.format(bird, sess, file, int(bout_s['start_sample']))
    
    #save as wav
    out_wav_path = os.path.join(bout_folder, 'bout_wav', out_fname)
    os.makedirs(os.path.split(out_wav_path)[0], exist_ok=True)
    #logger.info('saving to ' + out_wav_path)
    wavfile.write(out_wav_path, s_f, x)
    
    # make the bout_dict for the json file
    bout_len = x.size / s_f
    bout_dict = {'lenght_s': bout_len,
                'samplerate_hz': s_f,
                'wav_loc': out_wav_path,
                'indvs': {bird: 'European Starling'}}
    json_fname = '{}-{}-{}-{}.json'.format(bird, sess, file, int(bout_s['start_sample']))
    bout_json_path = os.path.join(bout_folder, 'bout_wav', json_fname)
    bout_to_json(bout_dict, bout_json_path)
    return out_wav_path
    
## these come from the params pickle file
s_f = 48000
border_ms = 900

bout_to_wav(one_bout, s_f=s_f, border_ms=border_ms)


'/mnt/sphere/speech_bci/analysis/bout/s_b1253_21/bout_wav/s_b1253_21-2021-06-06-07-00-01-01-10780560.wav'

In [59]:
## save all
only_bout_pd.apply(lambda s: bout_to_wav(s, s_f=s_f, border_ms=border_ms), axis=1)

0       /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
1       /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
2       /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
3       /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
4       /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
                              ...                        
2129    /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
2130    /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
2131    /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
2132    /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
2133    /mnt/sphere/speech_bci/analysis/bout/s_b1253_2...
Length: 1847, dtype: object